In [10]:
import pandas as pd

from plotly.offline import init_notebook_mode
from sqlalchemy import create_engine

from query_flow.parsers.postgres_parser import PostgresParser
from query_flow.vizualizers.query_vizualizer import QueryVizualizer

In [11]:
%load_ext autoreload
%autoreload 2

init_notebook_mode(connected=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
con_str = 'postgresql:///etrabelsi_thesis'
query_renderer = QueryVizualizer(parser=PostgresParser())

## Identifying duplications

In [13]:
query = """
SELECT *
FROM (SELECT *,
             row_number() OVER (PARTITION BY titles.title_id) as row_num
      FROM titles
      INNER JOIN generes
              ON generes.genere_name LIKE '%'|| titles.genres || '%'
      WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2 AND
            generes.kid_safe = True
     ) as t
WHERE row_num = 1
"""

In [14]:
query = """
SELECT *
FROM titles
INNER JOIN generes
        ON position(generes.genere_name in titles.genres) > 0
WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2 AND
      generes.kid_safe = True
"""

In [15]:
cardinality_df = query_renderer.get_cardinality_df(query, con_str)
query_renderer.vizualize(cardinality_df, title="Identifying duplications", metrics=["actual_rows"], open_=False)

In [16]:
query = """
SELECT *
FROM (SELECT *,
             row_number() OVER (PARTITION BY titles.title_id) as row_num
      FROM titles
      INNER JOIN generes
              ON generes.genere_name LIKE '%'|| titles.genres || '%'
      WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2  AND
            generes.kid_safe = True
     ) as t
WHERE row_num = 1
"""

In [17]:
query = """
SELECT *
FROM (SELECT *,
             row_number() OVER (PARTITION BY titles.title_id) as row_num
      FROM titles
      INNER JOIN generes
              ON position(generes.genere_name in titles.genres) > 0
      WHERE titles.premiered >= date_part('year', CURRENT_DATE) - 2  AND
            generes.kid_safe = True
     ) as t
WHERE row_num = 1
"""

In [22]:
cardinality_df = query_renderer.get_cardinality_df(query, con_str)
query_renderer.vizualize(cardinality_df, title="Identifying duplications", metrics=["actual_rows"], open_=False)